In [1]:
import torch
import torchvision

In [2]:
import torch.nn as nn

In [3]:
class AlexNetWithSoftmaxLayer(nn.Module):
    def __init__(self, original_model):
        super(AlexNetWithSoftmaxLayer, self).__init__()
        self.features = original_model.features
        self.classifier = original_model.classifier
        self.softmax = nn.Softmax(dim=1)  # softmax 레이어 추가

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = self.softmax(x)  # softmax 적용
        return x

In [4]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 7)

In [5]:
model = AlexNetWithSoftmaxLayer(model)

In [6]:
model.load_state_dict(torch.load('best_model_colab.pth'))

<All keys matched successfully>

In [7]:
device = torch.device('cuda')
model = model.to(device)

In [8]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [9]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
# from jetbot import Camera, bgr8_to_jpeg
from jetbot import bgr8_to_jpeg
from custom_camera import Camera

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
# blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image]))

In [10]:
from jetbot import Robot

robot = Robot()

In [11]:
import torch.nn.functional as F
import time

def update(change):
    global robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    #y = F.softmax(y, dim=1)
    max_index = torch.argmax(y)
    
    if max_index == 0:
        camera.text = "GOOD"
    elif max_index == 1:
        camera.text = "LEFT"
    elif max_index == 2:
        camera.text = "RIGHT"
    elif max_index == 3:
        camera.text = "TOP"
    elif max_index == 4:
        camera.text = "DOWN"
    elif max_index == 5:
        camera.text = "FAR"
    elif max_index == 6:
        camera.text = "NEAR"
    
        
#     time.sleep(1)
        
update({'new': camera.value})  # we call the function once to intialize

In [12]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [13]:
camera.unobserve(update, names='value')

Near
Near
Near
Near
